# EXPONENTIAL MODEL CATALOGE - for monkey Kate

In [37]:
warnings.filterwarnings("ignore")

In [38]:
import pickle
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from paths import Paths
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import warnings
from sklearn import metrics
import scipy as sp
from scipy.special import softmax
from scipy.optimize import minimize
from bandits_helper_functions import*
from exponential_model import Exponential_model, Optimize_exponent
from exponential_model import*
import pickle as pkl
import datetime
from openpyxl import load_workbook

## Define paths, import data

In [39]:
paths = Paths()
fig_save_path = paths.fig_save_path + 'exponential_model/'
data = pd.read_excel(paths.data + 'behavioural_data.xlsx', index_col=[0], converters={'session': str})

## Data formatting
- Drop the trials where the monkey didn't make any choice
- Keep only a smaller subset with the relevant columns

In [40]:
data = data.loc[~data['feedback'].isnull()]
data_subset = data[['monkey', 'session', 'trial_id', 'block_id', 'best_target', 'target', 'feedback']]

## Split the data according to the subject

In [41]:
ka_data = data_subset.loc[data_subset['monkey'] == 'ka']

In [42]:
ka_data = get_shift_stay_variables(ka_data)

## Assign alternative feedback coding columns

In [43]:
ka_data.loc[ka_data["feedback"] == 0, 'feedback_11'] = -1
ka_data.loc[ka_data["feedback"] == 1, 'feedback_11'] = 1
#ka_data_subset[['feedback', 'feedback_11']]

#### Get the previous traials' feedback history & Calculate choice match weight (CMW)

In [44]:
ka_data_fb_01_cmw_11 = get_n_back(ka_data, reward_code='feedback')

In [45]:
ka_data_fb_01_cmw_11 = get_binary_choice_match_value(ka_data_fb_01_cmw_11, coding = '-11')

----

# MODEL CATALOGE

# Section 1.: Reward coding: {0, 1}

## Feedback Source Inensitive (FSI)  model with reward coding {0, 1}

### KA_FSI_01

In [46]:
KA_FSI_01 = Optimize_exponent()
KA_FSI_01.optimise_model(data = ka_data_fb_01_cmw_11, name = 'KA_FSI_01',
                         FSS = False, fit_a2 = False, fit_eps = False, 
                         bounds = ((None, None), (0, 0), (-3, 1), (None, None), (0, 0)))

print_model_params(KA_FSI_01)
#save_model_results(KA_FSI_01)
#KA_FSI_01.save_data()

###################
Model name: KA_FSI_01
###################
Fitted params: {'a': 711.4001011594199, 'a2': None, 'b': -0.5486829803258474, 'c': -117.18603456597167, 'eps': None}
reward coding: {0.0, 1.0}
cmw coding: {1.0, -1.0}
###################
Log loss: 0.18352
###################


## Feedback Source Sensitive (FSS)  model with reward coding {0, 1}

### KA_FSS_01

In [47]:
KA_FSS_01 = Optimize_exponent()
KA_FSS_01.optimise_model(data = ka_data_fb_01_cmw_11, name = 'KA_FSS_01',
                         FSS = True, fit_a2 = False, fit_eps = False, 
                         bounds = ((None, None), (0, 0), (-3, 1), (None, None), (0, 0)))

print_model_params(KA_FSS_01)
#save_model_results(KA_FSS_01)
#KA_FSS_01.save_data()

###################
Model name: KA_FSS_01
###################
Fitted params: {'a': 1540.1489148485537, 'a2': None, 'b': -0.6786541652577511, 'c': -14.078573480439651, 'eps': None}
reward coding: {0.0, 1.0}
cmw coding: {1.0, -1.0}
###################
Log loss: 0.08932
###################


### KA_FSS_01_a2

In [48]:
KA_FSS_01_a2 = Optimize_exponent()
KA_FSS_01_a2.optimise_model(data = ka_data_fb_01_cmw_11, name = 'KA_FSS_01_a2',
                         FSS = True, fit_a2 = 'independent', fit_eps = False, 
                         bounds = ((None, None), (None, None), (-3, 1), (None, None), (0, 0)))

print_model_params(KA_FSS_01_a2)
#save_model_results(KA_FSS_01_a2)
KA_FSS_01_a2.save_data()

###################
Model name: KA_FSS_01_a2
###################
Fitted params: {'a': 1784.838139639883, 'a2': 643.5279320169332, 'b': -0.6191402012479515, 'c': -83.61938369819448, 'eps': None}
reward coding: {0.0, 1.0}
cmw coding: {1.0, -1.0}
###################
Log loss: 0.08356
###################


### KA_FSS_01_Pa2

In [49]:
KA_FSS_01_Pa2 = Optimize_exponent()
KA_FSS_01_Pa2.optimise_model(data = ka_data_fb_01_cmw_11, 
                             name = 'KA_FSS_01_Pa2',
                             FSS = True, 
                             fit_a2 = 'pseudo',
                             fit_eps = False, 
                             bounds = ((None, None), (0, 0), (-3, 1), (None, None), (0, 0)))

print_model_params(KA_FSS_01_Pa2)
#save_model_results(KA_FSS_01_Pa2)
#KA_FSS_01_Pa2.save_data()

###################
Model name: KA_FSS_01_Pa2
###################
Fitted params: {'a': 1586.780649345604, 'a2': -1586.780649345604, 'b': -0.8727760143106232, 'c': 51.69909362102566, 'eps': None}
reward coding: {0.0, 1.0}
cmw coding: {1.0, -1.0}
###################
Log loss: 0.11103
###################


### KA_FSS_eps1e2 (eps. upper= -0.01)

In [50]:
KA_FSS_eps1e2 = Optimize_exponent()
KA_FSS_eps1e2.optimise_model(data = ka_data_fb_01_cmw_11, name = 'KA_FSS_eps1e-2',
                         FSS = True, fit_a2 = False, fit_eps = True, 
                         bounds = ((None, None), (0, 0), (-3, 1), (None, None), (None, -0.01)))

print_model_params(KA_FSS_eps1e2)
#save_model_results(KA_FSS_eps1e2)
#KA_FSS_eps1e2.save_data()

###################
Model name: KA_FSS_eps1e-2
###################
Fitted params: {'a': 1216.6085385124707, 'a2': 0.0, 'b': -0.6104715405278982, 'c': -11.046951719032977, 'eps': -0.01}
reward coding: {1.0, -0.01}
cmw coding: {1.0, -1.0}
###################
Log loss: 0.09378
###################


### KA_FSS_eps1e3 (eps. upper= -0.001)

In [51]:
KA_FSS_eps1e3 = Optimize_exponent()
KA_FSS_eps1e3.optimise_model(data = ka_data_fb_01_cmw_11, name = 'KA_FSS_eps1e-3',
                         FSS = True, fit_a2 = False, fit_eps = True, 
                         bounds = ((None, None), (0, 0), (-3, 1), (None, None), (None, -0.001)))

print_model_params(KA_FSS_eps1e3)
#save_model_results(KA_FSS_eps1e3)
#KA_FSS_eps1e3.save_data()

###################
Model name: KA_FSS_eps1e-3
###################
Fitted params: {'a': 1495.698076209572, 'a2': 0.0, 'b': -0.6700707415695838, 'c': -13.750761496324419, 'eps': -0.001}
reward coding: {1.0, -0.001}
cmw coding: {1.0, -1.0}
###################
Log loss: 0.08979
###################


---

# Section 2: reward coding {-1, 1}

 Get the previous traials' feedback history & Calculate choice match weight (CMW)

In [52]:
ka_data_fb_11_cmw_01 = get_n_back(ka_data, reward_code='feedback_11')

In [53]:
ka_data_fb_11_cmw_01 = get_binary_choice_match_value(ka_data_fb_11_cmw_01, coding = '01')

## Feedback Source Inensitive (FSI)  model with reward coding {-1, 1}

### KA_FSI_11

In [54]:
KA_FSI_11 = Optimize_exponent()
KA_FSI_11.optimise_model(data = ka_data_fb_11_cmw_01, name = 'KA_FSI_11',
                         FSS = False, fit_a2 = False, fit_eps = False, 
                         bounds = ((None, None), (0, 0), (-3, 0), (None, None), (0, 0)))

print_model_params(KA_FSI_11)
#save_model_results(KA_FSI_11)
#KA_FSI_11.save_data()

###################
Model name: KA_FSI_11
###################
Fitted params: {'a': 372.52630958398197, 'a2': None, 'b': -0.5726359688897196, 'c': 368.28214373459986, 'eps': None}
reward coding: {1.0, -1.0}
cmw coding: {0.0, 1.0}
###################
Log loss: 0.18345
###################


## Feedback Source Sensitive (FSS)  model with reward coding {-1, 1}

### KA_FSS_11

In [55]:
KA_FSS_11 = Optimize_exponent()
KA_FSS_11.optimise_model(data = ka_data_fb_11_cmw_01, name = 'KA_FSS_11',
                         FSS = True, fit_a2 = False, fit_eps = False, 
                         bounds = ((None, None), (0, 0), (-3, 1), (None, None), (0, 0)))

print_model_params(KA_FSS_11)
#save_model_results(KA_FSS_11)
#KA_FSS_11.save_data()

###################
Model name: KA_FSS_11
###################
Fitted params: {'a': 55.49188332468125, 'a2': None, 'b': -0.0011430515721382651, 'c': 171.25832679413594, 'eps': None}
reward coding: {1.0, -1.0}
cmw coding: {0.0, 1.0}
###################
Log loss: 0.26401
###################


### KA_FSS_11_a2

In [56]:
KA_FSS_11_a2 = Optimize_exponent()
KA_FSS_11_a2.optimise_model(data = ka_data_fb_11_cmw_01, name = 'KA_FSS_11_a2',
                         FSS = True, fit_a2 = 'independent', fit_eps = False, 
                         bounds = ((None, None), (None, None), (-3, 1), (None, None), (0, 0)))

print_model_params(KA_FSS_11_a2)
#save_model_results(KA_FSS_11_a2)
#KA_FSS_11_a2.save_data()

###################
Model name: KA_FSS_11_a2
###################
Fitted params: {'a': 631.9557193292158, 'a2': 3859.746658752129, 'b': -1.632388661594395, 'c': 520.9594898626217, 'eps': None}
reward coding: {1.0, -1.0}
cmw coding: {0.0, 1.0}
###################
Log loss: 0.05787
###################


### KA_FSS_11_Pa2

In [57]:
KA_FSS_11_Pa2 = Optimize_exponent()
KA_FSS_11_Pa2.optimise_model(data = ka_data_fb_11_cmw_01, 
                             name = 'KA_FSS_11_Pa2',
                             FSS = True, 
                             fit_a2 = 'pseudo',
                             fit_eps = False, 
                             bounds = ((None, None), (0, 0), (-3, 0), (None, None), (0, 0)))

print_model_params(KA_FSS_11_Pa2)
#save_model_results(KA_FSS_11_Pa2)
#KA_FSS_11_Pa2.save_data()

###################
Model name: KA_FSS_11_Pa2
###################
Fitted params: {'a': 23.78845973509825, 'a2': -23.78845973509825, 'b': 0.0, 'c': 166.21537865788906, 'eps': None}
reward coding: {1.0, -1.0}
cmw coding: {0.0, 1.0}
###################
Log loss: 0.31278
###################
